Symbolic Regression: 
We can generalise the formula for the GBM to: 


$$ dS = f(S)dt + g(S)dM$$

<br> We can use Symbolic Regression to describe what $f$ and $g$ are. 

$$S_t = S_0 exp [{(\mu - \frac{\sigma^2}{2})t + \sigma W_t}]$$



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gplearn.genetic import SymbolicRegressor
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sympy import *
from sklearn.utils.random import check_random_state
import graphviz
import time
import yfinance as yf
from pysr import PySRRegressor

ModuleNotFoundError: No module named 'gplearn'

In [4]:
ticker_appl = "AAPL"

ticker = yf.Ticker(ticker_appl)

hist_data = ticker.history("1y")

hist_closing = np.array(hist_data["Close"])

#Calculate daily returns
T =  1
dS = np.diff(hist_closing)

drift_approx_target = dS/ T
diffusion_approx = (dS **2) / T 


NameError: name 'yf' is not defined

In [16]:
default_pysr_param = dict(
    populations = 30, 
    model_selection="best"
)

https://www.scribd.com/document/660463529/On-Numerical-Methods-for-Stochastic-SINDy-2023-25
<br>https://arxiv.org/html/2306.17814v2
<br>https://medium.com/@polanitzer/estimating-the-parameters-for-a-geometric-brownian-motion-stochastic-process-using-two-different-6c7cbdf20c8f


In [18]:
drift_approx = np.array(drift_approx_target.reshape(-1,1))

model_miu = PySRRegressor(
    niterations=40,  # < Increase me for better results
    binary_operators=["+", "*", "/", "-"],
    unary_operators=[
        "log",
        "exp",
        "sqrt"
    ],
    elementwise_loss="loss(prediction, target) = (prediction - target)^2",
    **default_pysr_param,
)
model_miu.fit(drift_approx, diffusion_approx)

C:\Users\jayma\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pysr\sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
Compiling Julia backend...
[IPKernelApp] ERROR | Error in message handler
Traceback (most recent call last):
  File "c:\Users\jayma\anaconda3\Lib\site-packages\ipykernel\kernelbase.py", line 545, in dispatch_queue
    await self.process_one()
  File "c:\Users\jayma\anaconda3\Lib\site-packages\ipykernel\kernelbase.py", line 534, in process_one
    await dispatch(*args)
  File "c:\Users\jayma\anaconda3\Lib\site-packages\ipykernel\kernelbase.py", line 393, in dispatch_shell
    idents, msg = self.session.feed_identities(msg, copy=False)
                  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^
  File "c:\Users\jayma\anaconda3\Lib\site-packages\jupyter_client\session.py", line 994, in feed_identities
    raise ValueEr

───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           3.251e+03  0.000e+00  y = 18.667
3           0.000e+00  5.164e+01  y = x₀ * x₀
───────────────────────────────────────────────────────────────────────────────────────────────────


,model_selection,'best'
,binary_operators,"['+', '*', ...]"
,unary_operators,"['log', 'exp', ...]"
,expression_spec,None
,niterations,40
,populations,30
,population_size,27
,max_evals,None
,maxsize,30
,maxdepth,None
,warmup_maxsize_by,None


  - outputs\20251114_163722_YgvNOD\hall_of_fame.csv


In [2]:
diff_approx = np.array(diffusion_approx.reshape(-1,1))

model_sigma = PySRRegressor(
    niterations=40,  # < Increase me for better results
    binary_operators=["+", "*", "/", "-"],
    unary_operators=[
        "log",
        "exp",
        "sqrt"
    ],
    elementwise_loss="loss(prediction, target) = (prediction - target)^2",
    **default_pysr_param,
)
model_sigma.fit(diff_approx, drift_approx_target)

NameError: name 'np' is not defined

In [1]:
model_miu.sympy()
model_sigma.sympy()

NameError: name 'model_miu' is not defined

In [ ]:
def geometricBrownianMotion(S0, mu, sigma, numofPaths, T, timeSteps):
    dt = T/timeSteps

    # Browniam increments
    dW = np.random.normal(0, np.sqrt(dt), size=(numofPaths, timeSteps)).T

    #Calculation for each step
    drift = (mu - ((sigma**2)/2)) * dt
    diffusion = sigma * dW
    increments = np.exp(drift + diffusion)


    St = np.vstack([np.ones(numofPaths), increments]).cumprod(axis=0) * S0

    return St